In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import pickle as pkl

In [3]:
%run utils.ipynb
%run targets.ipynb

In [4]:
file = 'data/ETHBTC_1m_1519496760000_1549568220000.csv'

In [5]:
# Get Data
df = pd.read_csv(file)
df.shape
df.head(10)

(500000, 12)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  VOLUME  \
0  1519496760000  0.085541  0.085570  0.085407  0.085473  27.714   
1  1519496820000  0.085473  0.085570  0.085448  0.085449  20.128   
2  1519496880000  0.085448  0.085569  0.085400  0.085538  45.510   
3  1519496940000  0.085538  0.085546  0.085402  0.085540  22.231   
4  1519497000000  0.085539  0.085552  0.085464  0.085464  15.243   
5  1519497060000  0.085466  0.085555  0.085464  0.085545  31.895   
6  1519497120000  0.085545  0.085545  0.085464  0.085539  13.325   
7  1519497180000  0.085539  0.085555  0.085414  0.085517  28.202   
8  1519497240000  0.085460  0.085545  0.085419  0.085534  15.302   
9  1519497300000  0.085440  0.085545  0.085440  0.085533  13.730   

      CLOSE_TIME  QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519496819999   2.370345         192         15.308        1.309733      0  
1  1519496879999   1.720609         122          6.579        0.562654      0  
2  1519496939999   3.890176         209         19.426        1.660960      0  
3  1519496999999   1.900513         105         14.320        1.224483      0  
4  1519497059999   1.303412         117          8.373        0.716219      0  
5  1519497119999   2.727413         131         19.237        1.645547      0  
6  1519497179999   1.139346          78          7.311        0.625352      0  
7  1519497239999   2.411093         195         14.409        1.232346      0  
8  1519497299999   1.307959         134          7.216        0.617035      0  
9  1519497359999   1.173580         105          6.186        0.529001      0

In [6]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.loc[:, ['OPEN', 'HIGH', 'LOW', 'CLOSE']] = data.loc[:, ['OPEN', 'HIGH', 'LOW', 'CLOSE']].diff()
data.shape
data[:5]

(500000, 5)

OPEN_TIME      OPEN          HIGH       LOW     CLOSE
0  1519496760000       NaN           NaN       NaN       NaN
1  1519496820000 -0.000068  0.000000e+00  0.000041 -0.000024
2  1519496880000 -0.000025 -1.000000e-06 -0.000048  0.000089
3  1519496940000  0.000090 -2.300000e-05  0.000002  0.000002
4  1519497000000  0.000001  6.000000e-06  0.000062 -0.000076

In [7]:
data_validation(data)

In [8]:
# Get Target
%run targets.ipynb
target = get_target_1(file)
target.shape
target.head()

(499999, 2)

OPEN_TIME  TARGET
0  1519496760000     1.0
1  1519496820000     0.0
2  1519496880000     0.0
3  1519496940000     1.0
4  1519497000000     0.0

In [9]:
data = pd.merge(data, target, on='OPEN_TIME')
data.shape
data.head()

(499999, 6)

OPEN_TIME      OPEN          HIGH       LOW     CLOSE  TARGET
0  1519496760000       NaN           NaN       NaN       NaN     1.0
1  1519496820000 -0.000068  0.000000e+00  0.000041 -0.000024     0.0
2  1519496880000 -0.000025 -1.000000e-06 -0.000048  0.000089     0.0
3  1519496940000  0.000090 -2.300000e-05  0.000002  0.000002     1.0
4  1519497000000  0.000001  6.000000e-06  0.000062 -0.000076     0.0

In [10]:
data_validation(data)

In [11]:
data = data.iloc[1:]

In [12]:
data.describe()

OPEN_TIME          OPEN          HIGH           LOW         CLOSE  \
count  4.999980e+05  4.999980e+05  4.999980e+05  4.999980e+05  4.999980e+05   
mean   1.534528e+12 -1.096064e-07 -1.096624e-07 -1.093484e-07 -1.094784e-07   
std    8.686669e+09  4.246312e-05  4.064354e-05  4.323908e-05  4.440037e-05   
min    1.519497e+12 -1.167000e-03 -4.428000e-03 -3.904000e-03 -9.980000e-04   
25%    1.526997e+12 -1.500000e-05 -1.100000e-05 -9.000000e-06 -1.600000e-05   
50%    1.534530e+12  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
75%    1.542043e+12  1.500000e-05  1.000000e-05  1.200000e-05  1.500000e-05   
max    1.549568e+12  1.190000e-03  5.215000e-03  3.786000e-03  1.176000e-03   

              TARGET  
count  499998.000000  
mean        0.479844  
std         0.499594  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         1.000000  
max         1.000000

In [13]:
frac_data_use = 0.2

tr_frac = 0.6
cv_frac = 0.2
tt_frac = 0.2

TR_data = data[:int(len(data)*tr_frac)]
CV_data = data[int(len(data)*tr_frac):int(len(data)*(tr_frac+cv_frac))]
TT_data = data[int(len(data)*(tr_frac+cv_frac)):]

TR_data.shape
CV_data.shape
TT_data.shape

(299998, 6)

(100000, 6)

(100000, 6)

In [14]:
# Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler = scaler.fit(TR_data.iloc[:, 1:-1].values)

TR_data.iloc[:, 1:-1] = scaler.transform(TR_data.iloc[:, 1:-1])
CV_data.iloc[:, 1:-1] = scaler.transform(CV_data.iloc[:, 1:-1])
TT_data.iloc[:, 1:-1] = scaler.transform(TT_data.iloc[:, 1:-1])

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [71]:
win=900
TR_X, TR_y = get_multiple_ts_dataset(TR_data.values, tr_win=win, tt_win=0, point_target=True)
CV_X, CV_y = get_multiple_ts_dataset(CV_data.values, tr_win=win, tt_win=0, point_target=True)
TT_X, TT_y = get_multiple_ts_dataset(TT_data.values, tr_win=win, tt_win=0, point_target=True)

In [72]:
TR_X.shape
TR_y.shape

(299099, 3601)

(299099,)

In [ ]:
from sklearn.linear_model import LogisticRegression

params = {
            'dual'              : False,
            'fit_intercept'     : True,
            'intercept_scaling' : 1,
            'max_iter'          : 100,
            'multi_class'       : 'ovr',
            'class_weight'      : None,
            'random_state'      : 0,
            'verbose'           : 1,
            'warm_start'        : False,
            'n_jobs'            : -1,
            'penalty'           : 'l1',
            'solver'            : 'saga'
        }

params['C'] = 30
params['tol'] = 0.00001

model = LogisticRegression(**params)
model = model.fit(TR_X[:, 1:], TR_y)

TR_y_pred = model.predict_proba(TR_X[:, 1:])[:, 1]
CV_y_pred = model.predict_proba(CV_X[:, 1:])[:, 1]
TT_y_pred = model.predict_proba(TT_X[:, 1:])[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(CV_metrics, name='CV'))
res = res.append(pd.Series(TT_metrics, name='TT'))

print(res)

In [65]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.plot(model.coef_[0,:])

In [66]:
TR_y_pred = model.predict(TR_X[:, 1:])
TR_y_pred[:5]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TR_metrics

array([0., 1., 1., 0., 0.])

{'acc': 0.5342627102526202,
 'auc': 0.5242664173835343,
 'f1': 0.36375073502263183,
 'log_loss': 16.086091362351826}

In [67]:
(TR_y_pred > 0.5).sum()

75602

In [68]:
CV_y_pred = model.predict(CV_X[:, 1:])
CV_y_pred[:5]

CV_metrics = eval_class_metrics(CV_y, CV_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
CV_metrics

array([0., 0., 0., 0., 0.])

{'acc': 0.5214892528660695,
 'auc': 0.5013708498988343,
 'f1': 0.13534870233434826,
 'log_loss': 16.527207488917988}

In [69]:
TT_y_pred = model.predict(TT_X[:, 1:])
TT_y_pred[:5]

TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics

array([0., 0., 0., 0., 0.])

{'acc': 0.5162535982587938,
 'auc': 0.5030658628914745,
 'f1': 0.15480863591756625,
 'log_loss': 16.708044041259875}

In [70]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL', 
    'diff':True,
    'rollNorm':False,
    'win':win
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': 'LRL1',
    'accuracy':'{:.4f}'.format((CV_metrics['acc']+TT_metrics['acc'])/2)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

f = open('models/'+time_tag+'.pkl', 'wb')
pkl.dump(model, f, -1)
f.close()

%notebook -e models/{big_tag}.ipynb